# Part 1：PYTORCH C++ API的组成

截止到PyTorch 1.7.1版本，其C++ API由5大部分组成，如下表所示：

| 命名空间 | 功能描述  |
| :----: | :---- |
| **ATen**   | The foundational tensor and mathematical operation library on which all else is built. |
| **Autograd**   | Augments ATen with automatic differentiation. |
| **C++ Frontend**   | High level constructs for training and evaluation of machine learning models. |
| **TorchScript**   | An interface to the TorchScript JIT compiler and interpreter. |
| **C++ Extensions**   | A means of extending the Python API with custom C++ and CUDA routines. |


## ATen
ATen是一个基础的tensor库，PyTorch中Python和C++接口函数都是构建在这个库的基础上的。它提供了一个Tensor类，在这个类中定义了上百中tensor的相关操作，多数操作可以同时支持CPU和GPU实现，Tensor类可以根据数据类型来动态指定在哪种处理器上执行操作。其命名空间为at::。下面是个简单的例子：

In [1]:
#include <iostream>
#include <vector>

/* a workaround to solve cling issue */
#include "../macos_cling_workaround.hpp"
/*set libtorch path, load libs*/
#include "../load_libtorch.hpp"
#include <torch/torch.h>
#include <ATen/ATen.h>

at::Tensor a = at::ones({2, 2}, at::kInt);
std::cout << "a = " << std::endl << a << std::endl;
at::Tensor b = at::randn({2, 2}, at::kFloat);
std::cout << "b = " << std::endl << b << std::endl;
auto c = a + b.to(at::kInt);
std::cout << "c = " << std::endl << c << std::endl;

a = 
 1  1
 1  1
[ CPUIntType{2,2} ]
b = 
 1.2360 -0.8318
 0.3336  0.8639
[ CPUFloatType{2,2} ]
c = 
 2  1
 1  1
[ CPUIntType{2,2} ]


## Autograd
autograd是PyTorch C++ API的一部分，属于对ATen的Tensor类的功能增强，使其具备自动化微分功能。autograd系统会记录tensor上的操作，进而形成一个autograd graph。调用tensor变量的backwards()函数会产生逆向的差分运算。
注意：ATen中的at::Tensor类默认并不具备微分功能。为了使tensors具备可微分功能，你必须使用tensor的工厂方法，即调用torch::命名空间的函数，而不是去直接调用at::命名空间的函数。比如，at::ones()函数产生的tensor是不具备可微分功能的，但是torch::ones()函数的就可以。

## C++ Frontend
PyTorch C++前端框架为神经网络和通用机器学习提供了一套纯C++的模型接口，包括：
* 用于自定义机器学习模型的层级模型接口系统；
* 为构建深度学习模型的常见函数库，如卷积运算，RNN，BN等；
* 深度学习优化方法API，如SGD,Adam,RMSprop等；
* 一种数据集和数据管道的表示方式；
* 序列化存储和加载训练过程中checkpoints的格式；
* 在多GPU上并行运行模型；
* TorchScript JIT compiler的接入口；

## TorchScript
这是对PyTorch模型的一种表示方法。事实上，TorchScript在某种意义上是一种编程语言，是Python的一个子集。TorchScript的C++接口包含的主要功能有：
* 加载和运行序列化TorchScript模型；
* 提供相关API，使用户可以自定义算子；
* TorchScript C++程序的即时编译；

对于第一个功能而言，这个给用户提供了一种方法，即开发者可以使用python来编写算法并训练，训练得到的模型参数可以直接用于C++程序中，而无需使用libtorch重新编写深度学习模型并从头训练。（听着不错，但实测发现有某种程度的精度损失）。

## C++ Extensions
C++扩展提供了一种简单而有效的方法，使得开发者可以方便的访问PyTorch的上层接口。其最常见的用法是让开发者可以自定义算子。